In [35]:
from theano.sandbox import cuda

In [36]:
from imp import reload
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [37]:
#path = "/home/jd/aidata/ml-20m/"

model_path = '/home/jd/aidata/imdb/'
from IPython.display import Audio, display
from imp import reload
import time

def allDone():
 #   self.showtraceback((etype, value, tb), tb_offset=tb_offset)
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

allDone()

In [1]:
from IPython.display import Audio, display
def allDone():
 #   self.showtraceback((etype, value, tb), tb_offset=tb_offset)
  display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

In [2]:
allDone()

## Ready to load

In [38]:
## the model is rating and is loaded into ratings
## pd stand for panda
ratings = pd.read_csv(path+'ratings.csv')
## This prints off the top 5
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [39]:
# The number of ratings
len(ratings)

100004

In [40]:
# add in the names
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [41]:
# ratings for each section of ratings
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [42]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [43]:
print (movies)

[  31 1029 1061 ...,  129 4736 6425]


In [44]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [45]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
#Prints out array of users min number there and the max
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [46]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

## The number of latent factors

In [47]:
n_factors = 50

In [48]:
np.random.seed = 42

Randomly split into training and validation.

In [49]:
# Split everything into the trn and validation data
msk = np.random.rand(len(ratings)) < 0.8
# everything is divided
trn = ratings[msk]
val = ratings[~msk]

## Subset of excel file

Used to create something you can easily drop into excel

In [50]:
## create a subset for users
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [51]:
## Also a subset for ratings
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:15]

In [52]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [53]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [54]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product


In [55]:
# Embedding is ?
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
#
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [56]:
# Put users and movies together
x = merge([u, m], mode='dot')
#  What is flatten
x = Flatten()(x)
model = Model([user_in, movie_in], x)
# MSE = categorical_crossentropy
# Adam helps decide learning rate
model.compile(Adam(0.001), loss='mse')

In [57]:
# Tries to fit the model
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/4
80024/80024 [==============================] - 4s - loss: 9.9701 - val_loss: 4.3601
Epoch 2/4
80024/80024 [==============================] - 4s - loss: 3.1324 - val_loss: 2.8531
Epoch 3/4
80024/80024 [==============================] - 4s - loss: 2.4113 - val_loss: 2.6412
Epoch 4/4
80024/80024 [==============================] - 4s - loss: 2.2476 - val_loss: 2.5893


In [58]:
# change learning rate
model.optimizer.lr=0.01

In [59]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/4
80024/80024 [==============================] - 4s - loss: 2.1855 - val_loss: 2.5749
Epoch 2/4
80024/80024 [==============================] - 4s - loss: 2.1522 - val_loss: 2.5721
Epoch 3/4
80024/80024 [==============================] - 4s - loss: 2.1246 - val_loss: 2.5777
Epoch 4/4
80024/80024 [==============================] - 4s - loss: 2.0998 - val_loss: 2.5676


In [60]:
model.optimizer.lr=0.001

In [61]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=4, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/4
80024/80024 [==============================] - 4s - loss: 2.0774 - val_loss: 2.5730
Epoch 2/4
80024/80024 [==============================] - 4s - loss: 2.0546 - val_loss: 2.5841
Epoch 3/4
80024/80024 [==============================] - 4s - loss: 2.0345 - val_loss: 2.5887
Epoch 4/4
80024/80024 [==============================] - 4s - loss: 2.0131 - val_loss: 2.6004


Model doesnt work well because of bias

## Correcting BIAS

In [63]:
# 
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [64]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-4)

In [65]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [66]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [67]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [68]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/1
80024/80024 [==============================] - 3s - loss: 8.7908 - val_loss: 3.5235


In [69]:
model.optimizer.lr=0.01

In [70]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/6
80024/80024 [==============================] - 3s - loss: 2.5670 - val_loss: 2.3325
Epoch 2/6
80024/80024 [==============================] - 3s - loss: 2.0007 - val_loss: 2.1256
Epoch 3/6
80024/80024 [==============================] - 3s - loss: 1.8486 - val_loss: 2.0308
Epoch 4/6
80024/80024 [==============================] - 3s - loss: 1.7568 - val_loss: 1.9548
Epoch 5/6
80024/80024 [==============================] - 3s - loss: 1.6778 - val_loss: 1.8793
Epoch 6/6
80024/80024 [==============================] - 3s - loss: 1.6025 - val_loss: 1.8105


In [71]:
model.optimizer.lr=0.001

In [72]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=10, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/10
80024/80024 [==============================] - 3s - loss: 1.5270 - val_loss: 1.7422
Epoch 2/10
80024/80024 [==============================] - 3s - loss: 1.4542 - val_loss: 1.6821
Epoch 3/10
80024/80024 [==============================] - 3s - loss: 1.3843 - val_loss: 1.6248
Epoch 4/10
80024/80024 [==============================] - 3s - loss: 1.3160 - val_loss: 1.5700
Epoch 5/10
80024/80024 [==============================] - 3s - loss: 1.2516 - val_loss: 1.5189
Epoch 6/10
80024/80024 [==============================] - 3s - loss: 1.1903 - val_loss: 1.4699
Epoch 7/10
80024/80024 [==============================] - 3s - loss: 1.1315 - val_loss: 1.4226
Epoch 8/10
80024/80024 [==============================] - 3s - loss: 1.0753 - val_loss: 1.3785
Epoch 9/10
80024/80024 [==============================] - 3s - loss: 1.0224 - val_loss: 1.3392
Epoch 10/10
80024/80024 [==============================] - 3s - loss: 0.9730 - val_loss: 1.3033

In [73]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80024 samples, validate on 19980 samples
Epoch 1/5
80024/80024 [==============================] - 3s - loss: 0.9265 - val_loss: 1.2683
Epoch 2/5
80024/80024 [==============================] - 3s - loss: 0.8823 - val_loss: 1.2363
Epoch 3/5
80024/80024 [==============================] - 3s - loss: 0.8423 - val_loss: 1.2050
Epoch 4/5
80024/80024 [==============================] - 3s - loss: 0.8042 - val_loss: 1.1794
Epoch 5/5
80024/80024 [==============================] - 3s - loss: 0.7704 - val_loss: 1.1548


In [74]:
model.save_weights(model_path+'bias.h5')

In [75]:
model.load_weights(model_path+'bias.h5')

In [76]:
model.predict([np.array([3]), np.array([6])])

array([[ 4.8921]], dtype=float32)

## Analyze results